#### CV Libraries
* torchvision
* torchvision.datasets
* torchvision.models
* torchvision.transforms

In [6]:
import os
import torch
import torchvision
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [8]:
DATA_DIR = 'data/'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [9]:
# get dataset fashion MNIST for testing
train_data = datasets.FashionMNIST(root=DATA_DIR,
                                   train=True,
                                   download=True,
                                   transform=ToTensor(),
                                   target_transform=None)

test_data = datasets.FashionMNIST(root=DATA_DIR,
                                  train=False,
                                  download=True,
                                  transform=ToTensor(),
                                  target_transform=None)

0.4%

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%
0.7%

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
len()